### Beispielskript zur Nutzung von Neo4j über Python

#### 1. Installieren und importieren aller relevanten Packages

In [ ]:
import pandas as pd
from neo4j import GraphDatabase

#### 2. Erstellen eine Verbindung zur Neo4j Datenbank

In [ ]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [ ]:
# Passe die Parameter an die erstellte Datenbank an
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="drugshortage")

#### 3. Importieren der Excel oder csv Datei

In [ ]:
# Spezifiziere den Pfad der Datei
drug_shortages_path = 'C:/Users/usena/Documents/Drug_Shortages_Dataset.xlsx'

# Importiere die Datei (hier bspw. in Form eines Pandas Dataframes https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)
drug_shortages = pd.read_excel(drug_shortages_path)
drug_shortages.head()

#### 4. Verarbeitung der Daten

In [ ]:
# Die Daten können nun in Python verarbeiten werden (bspw. bereinigen, filtern, übersetzen, etc.)
# Alternativ kann auch ein bereits bearbeiteter Datensatz eingelesen werden 
# Dieser Schritt ist sehr relevant 

#### 5. Erstellen von Knoten 

In [ ]:
# Beispielsweise gibt es die Möglichkeit mit Constraints zu arbeiten
# Erstelle von Constaints, um sicher zu gehen, dass jeder Knoten nur einmal auftritt
constraint_drug = "CREATE CONSTRAINT ON (d:drug) ASSERT d.name IS UNIQUE"
conn.query(constraint_drug)

constraint_substance = "CREATE CONSTRAINT ON (s:substance) ASSERT s.name IS UNIQUE"
conn.query(constraint_substance)

In [ ]:
# Beispielhafte Erstellung von Knoten der Wirkstoffe (hier als 'substance' bezeichnet)
for index, row in drug_shortages.iterrows():
    # festlegen der Properties
    atc_code = row['Atc Code']  
    sub_name = row['Wirkstoffe']  
    # erstellen der Knoten n vom Typ 'substance' mit properties
    query = f"CREATE (n:substance{{atc_code: '{atc_code}', name: '{sub_name}'}})"
    conn.query(query)

In [ ]:
# Beispielhafte Erstellung von Knoten der Arzneimittel (hier als 'drug' bezeichnet)
for index, row in drug_shortages.iterrows():
    drug_name = row['Arzneimittlbezeichnung']  
    query = f"CREATE (n:drug{{name: '{drug_name}'}})"
    conn.query(query)

#### 6. Erstellen von Verbindungen

In [ ]:
# Beispielhafte Erstellung einer Verbindung zwischen den Knoten "Drug" und "Substance"
for index, row in drug_shortages.iterrows():
    drug_name = row['Arzneimittlbezeichnung']  
    atc_code = row['Atc Code'] 
    
    # Cypher-Abfrage zum Erstellen der Beziehung
    query = f"MATCH (d:drug {{name: '{drug_name}'}}), (s:substance {{atc_code: '{atc_code}'}}) CREATE (d)-[:HAS_SUBSTANCE]->(s)"
    conn.query(query)